In [1]:
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import hopsworks
import pandas as pd

In [3]:
# from sklearn.neighbors import KNeighborsClassifier
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from matplotlib import pyplot
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib
import os

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sodapy import Socrata
# Load the data
client = Socrata("data.sfgov.org", "7cCZMHDK3CTLNu0K31rZxP1bl", username="zzp9067788@gmail.com", password="Zzp123456.")
results = client.get("ubvf-ztfx", limit=800000)
incident_df = pd.DataFrame.from_records(results)
# List of columns to keep
columns_to_keep = [
    'weather_1', 'collision_severity', 'type_of_collision', 
    'road_surface', 'road_cond_1', 'lighting', 
    'control_device', 'intersection', 'number_killed', 'number_injured','time_cat','mviw','ped_action','party1_type','party2_type','party1_move_pre_acc','party2_move_pre_acc'
]

# Select only the specified columns
incident_df_subset = incident_df[columns_to_keep]

# Now, incident_df_subset contains only the desired columns


In [5]:
incident_df_subset['collision_severity'].value_counts()
def remap_severity(severity):
    if severity == 'Injury (Complaint of Pain)':
        return 0  # Minor Injury
    elif severity == 'Injury (Other Visible)':
        return 1  # Moderate Injury
    else:
        return 2  # Severe Injury (including 'Injury (Severe)', 'Fatal', 'Medical')

incident_df_subset['collision_severity_class'] = incident_df_subset['collision_severity'].apply(remap_severity)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
incident_df_subset

,weather_1,collision_severity,type_of_collision,road_surface,road_cond_1,lighting,control_device,intersection,number_killed,number_injured,time_cat,mviw,ped_action,party1_type,party2_type,party1_move_pre_acc,party2_move_pre_acc,collision_severity_class
0,Clear,Injury (Severe),Hit Object,Dry,Other,Dark - Street Lights,None,Intersection <= 20ft,0,1,2:01 pm to 6:00 pm,Non-Collision,No Pedestrian Involved,Bicyclist,NaN,Other,NaN,2
1,Clear,Injury (Other Visible),Vehicle/Pedestrian,Dry,No Unusual Condition,Dark - Street Lights,None,Midblock > 20ft,0,1,6:01 pm to 10:00 pm,Pedestrian,Crossing Not in Crosswalk,Pedestrian,Driver,Other,Not Stated,1
2,Clear,Injury (Other Visible),Other,Dry,No Unusual Condition,Dark - Street Lights,None,Midblock > 20ft,0,2,10:01 pm to 2:00 am,Non-Collision,No Pedestrian Involved,Driver,NaN,Proceeding Straight,NaN,1
3,Clear,Injury (Other Visible),Rear End,Dry,No Unusual Condition,Daylight,None,Intersection Rear End <= 150ft,0,2,10:01 am to 2:00 pm,Parked Motor Vehicle,No Pedestrian Involved,Driver,Parked Vehicle,Other Unsafe Turning,Parked,1
4,Clear,Injury (Other Visible),Not Stated,Dry,No Unusual Condition,Daylight,None,Midblock > 20ft,0,1,10:01 am to 2:00 pm,Non-Collision,No Pedestrian Involved,Driver,NaN,Proceeding Straight,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57451,Raining,Injury (Complaint of Pain),Vehicle/Pedestrian,Slippery,No Unusual Condition,Dark - Street Lights,Functioning,Intersection <= 20ft,0,1,10:01 pm to 2:00 am,Pedestrian,Crossing Not in Crosswalk,Pedestrian,Driver,Proceeding Straight,Proceeding Straight,0
57452,Clear,Injury (Complaint of Pain),Other,Dry,No Unusual Condition,Daylight,None,Midblock > 20ft,0,1,2:01 pm to 6:00 pm,Not Stated,No Pedestrian Involved,Driver,NaN,Slowing/Stopping,NaN,0
57453,Clear,Injury (Other Visible),Sideswipe,Dry,No Unusual Condition,Dark - Street Lights,None,Midblock > 20ft,0,1,2:01 am to 6:00 am,Parked Motor Vehicle,No Pedestrian Involved,Driver,Parked Vehicle,Proceeding Straight,Not Stated,1
57454,Clear,Injury (Complaint of Pain),Broadside,Dry,No Unusual Condition,Daylight,None,Intersection <= 20ft,0,1,2:01 pm to 6:00 pm,Bicycle,No Pedestrian Involved,Bicyclist,Driver,Proceeding Straight,Making Left Turn,0


In [7]:
incident_df_subset['intersection'].value_counts()

intersection
Intersection <= 20ft              37698
Midblock > 20ft                   15806
Intersection Rear End <= 150ft     3951
Name: count, dtype: int64

In [8]:
incident_df_subset['road_cond_1'].value_counts()

road_cond_1
No Unusual Condition           54319
Not Stated                      1322
Other                            685
Construction or Repair Zone      480
Holes, Deep Ruts                 268
Obstruction on Roadway           127
Loose Material on Roadway        119
Holes, Deep Rut                   72
Reduced Roadway Width             49
Flooded                           15
Name: count, dtype: int64

In [9]:
incident_df_subset['control_device'].value_counts()

control_device
Functioning        37052
None               19369
Not Stated           827
Not Functioning      164
Obscured              44
Name: count, dtype: int64

In [10]:
incident_df_subset['lighting'].value_counts()

lighting
Daylight                                37252
Dark - Street Lights                    17122
Dusk - Dawn                              1992
Not Stated                                576
Dark - No Street Lights                   410
Dark - Street Lights Not Functioning      104
Name: count, dtype: int64

In [11]:
incident_df_subset['road_surface'].value_counts()

road_surface
Dry             50540
Wet              5790
Not Stated       1013
Slippery           79
Snowy or Icy       34
Name: count, dtype: int64

In [13]:
incident_df_subset['type_of_collision'].value_counts()

type_of_collision
Broadside             17524
Vehicle/Pedestrian    12171
Rear End               9353
Sideswipe              7473
Head-On                3397
Other                  2996
Hit Object             2191
Not Stated             1318
Overturned             1033
Name: count, dtype: int64

In [ ]:
incident_df_subset['mviw'].value_counts()

mviw
Other Motor Vehicle               26440
Pedestrian                        13576
Bicycle                            7875
Fixed Object                       2585
Parked Motor Vehicle               2394
Non-Collision                      1322
Not Stated                         1218
Other Object                       1054
Motor Vehicle on Other Roadway      833
Train                               136
Animal                               23
Name: count, dtype: int64

In [160]:
incident_df_subset['time_cat'].value_counts()

time_cat
2:01 pm to 6:00 pm     15300
10:01 am to 2:00 pm    11835
6:01 pm to 10:00 pm    11732
6:01 am to 10:00 am     9787
10:01 pm to 2:00 am     6121
2:01 am to 6:00 am      2629
Name: count, dtype: int64

In [161]:
incident_df_subset['party2_move_pre_acc'].value_counts()

party2_move_pre_acc
Proceeding Straight                       33515
Stopped In Road                            5098
Not Stated                                 2879
Making Left Turn                           2483
Parked                                     2324
Other                                      1606
Stopped                                    1533
Making Right Turn                          1363
Slowing/Stopping                            949
Entering Traffic                            376
Backing                                     354
Changing Lanes                              243
Parking Maneuver                            166
Passing Other Vehicle                       158
Making U Turn                               142
Merging                                      56
Ran Off Road                                 41
Traveling Wrong Way                          30
Crossed Into Opposing Lane                   21
Other Unsafe Turning                          9
Crossed Into Opposin

In [162]:
incident_df_subset['party1_move_pre_acc'].value_counts()

party1_move_pre_acc
Proceeding Straight                       29779
Making Left Turn                          10043
Making Right Turn                          3533
Changing Lanes                             1959
Other                                      1620
Entering Traffic                           1526
Not Stated                                 1305
Backing                                    1228
Stopped In Road                            1212
Making U Turn                              1036
Parked                                      705
Slowing/Stopping                            639
Passing Other Vehicle                       626
Traveling Wrong Way                         416
Stopped                                     378
Ran Off Road                                372
Other Unsafe Turning                        336
Parking Maneuver                            326
Crossed Into Opposing Lane                  179
Merging                                     150
Crossed Into Opposin

In [163]:
incident_df_subset['ped_action'].value_counts()

ped_action
No Pedestrian Involved                       42728
Crossing in Crosswalk at Intersection         8320
Crossing Not in Crosswalk                     2980
In Road, Including Shoulder                   1784
Not in Road                                    754
Not Stated                                     667
Crossing in Crosswalk Not at Intersection      200
Approaching/Leaving School Bus                  12
Not In Road                                     11
Name: count, dtype: int64

In [191]:
import os

import modal

stub = modal.Stub()


@stub.function(secret=modal.Secret.from_name("ID2223"))
def f():
    print(os.environ["HOPSWORKS_API_KEY"])

In [164]:
incident_df_subset['party1_type'].value_counts()

party1_type
Driver            47047
Bicyclist          4687
Pedestrian         4581
Other               763
Parked Vehicle      339
Not Stated           27
Bicycle               1
Name: count, dtype: int64

In [165]:
incident_df_subset['party2_type'].value_counts()

party2_type
Driver            35631
Pedestrian         9554
Bicyclist          5068
Parked Vehicle     2530
Other               549
Not Stated           19
Name: count, dtype: int64

In [166]:
incident_df_subset['mviw'].value_counts()

mviw
Other Motor Vehicle               26440
Pedestrian                        13576
Bicycle                            7875
Fixed Object                       2585
Parked Motor Vehicle               2394
Non-Collision                      1322
Not Stated                         1218
Other Object                       1054
Motor Vehicle on Other Roadway      833
Train                               136
Animal                               23
Name: count, dtype: int64

In [174]:
def map_weather_condition(condition):
    if condition in ['Clear', 'Cloudy', 'Raining', 'Not Stated']:
        return condition
    elif 'Fog' in condition:
        return 'Fog'
    elif condition in ['Wind', 'Snowing', 'Other', 'Other: NOT ON SCENE', 
                        'Other: Unknown', 'Other: NOT AT SCENE', 'Other: MISTING']:
        return 'Other'
    else:
        return 'Unknown'  # Fallback category

incident_df_subset['weather_1_grouped'] = incident_df_subset['weather_1'].apply(map_weather_condition)

# Convert the grouped categories to numerical values
weather_grouped_mapping = {label: idx for idx, label in enumerate(incident_df_subset['weather_1_grouped'].unique())}
incident_df_subset['weather_1_mapped'] = incident_df_subset['weather_1_grouped'].map(weather_grouped_mapping)

intersection_mapping = {label: idx for idx, label in enumerate(incident_df_subset['intersection'].unique())}
incident_df_subset['intersection_mapped'] = incident_df_subset['intersection'].map(intersection_mapping)

road_cond_mapping = {label: idx for idx, label in enumerate(incident_df_subset['road_cond_1'].unique())}
incident_df_subset['road_cond_1_mapped'] = incident_df_subset['road_cond_1'].map(road_cond_mapping)

control_device_mapping = {label: idx for idx, label in enumerate(incident_df_subset['control_device'].unique())}
incident_df_subset['control_device_mapped'] = incident_df_subset['control_device'].map(control_device_mapping)

lighting_mapping = {label: idx for idx, label in enumerate(incident_df_subset['lighting'].unique())}
incident_df_subset['lighting_mapped'] = incident_df_subset['lighting'].map(lighting_mapping)

road_surface_mapping = {label: idx for idx, label in enumerate(incident_df_subset['road_surface'].unique())}
incident_df_subset['road_surface_mapped'] = incident_df_subset['road_surface'].map(road_surface_mapping)

type_of_collision_mapping = {label: idx for idx, label in enumerate(incident_df_subset['type_of_collision'].unique())}
incident_df_subset['type_of_collision_mapped'] = incident_df_subset['type_of_collision'].map(type_of_collision_mapping)

time_cat_mapping = {label: idx for idx, label in enumerate(incident_df_subset['time_cat'].unique())}
incident_df_subset['time_cat_mapped'] = incident_df_subset['time_cat'].map(time_cat_mapping)

party2_move_mapping = {label: idx for idx, label in enumerate(incident_df_subset['party2_move_pre_acc'].unique())}
incident_df_subset['party2_move_pre_acc_mapped'] = incident_df_subset['party2_move_pre_acc'].map(party2_move_mapping)

party1_move_mapping = {label: idx for idx, label in enumerate(incident_df_subset['party1_move_pre_acc'].unique())}
incident_df_subset['party1_move_pre_acc_mapped'] = incident_df_subset['party1_move_pre_acc'].map(party1_move_mapping)

ped_action_mapping = {label: idx for idx, label in enumerate(incident_df_subset['ped_action'].unique())}
incident_df_subset['ped_action_mapped'] = incident_df_subset['ped_action'].map(ped_action_mapping)

party1_type_mapping = {label: idx for idx, label in enumerate(incident_df_subset['party1_type'].unique())}
incident_df_subset['party1_type_mapped'] = incident_df_subset['party1_type'].map(party1_type_mapping)

party2_type_mapping = {label: idx for idx, label in enumerate(incident_df_subset['party2_type'].unique())}
incident_df_subset['party2_type_mapped'] = incident_df_subset['party2_type'].map(party2_type_mapping)

mviw_mapping = {label: idx for idx, label in enumerate(incident_df_subset['mviw'].unique())}
incident_df_subset['mviw_mapped'] = incident_df_subset['mviw'].map(mviw_mapping)

# List of mapped columns to keep
mapped_columns = [
    'weather_1_mapped', 'intersection_mapped', 'road_cond_1_mapped', 
    'control_device_mapped', 'lighting_mapped', 'road_surface_mapped', 
    'type_of_collision_mapped', 'time_cat_mapped', 'party2_move_pre_acc_mapped', 
    'party1_move_pre_acc_mapped', 'ped_action_mapped', 'party1_type_mapped', 
    'party2_type_mapped', 'mviw_mapped','collision_severity_class'
]

# Create a new DataFrame with only the mapped columns
mapped_incident_df = incident_df_subset[mapped_columns]

# Now, 'mapped_incident_df' contains only the mapped versions of your original categorical variables


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [187]:
mapped_incident_df

,weather_1_mapped,intersection_mapped,road_cond_1_mapped,control_device_mapped,lighting_mapped,road_surface_mapped,type_of_collision_mapped,time_cat_mapped,party2_move_pre_acc_mapped,party1_move_pre_acc_mapped,ped_action_mapped,party1_type_mapped,party2_type_mapped,mviw_mapped,collision_severity_class
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,0,1,1,0,0,0,1,1,1,0,1,1,1,1,1
2,0,1,1,0,0,0,2,2,0,1,0,2,0,0,1
3,0,2,1,0,1,0,3,3,2,2,0,2,2,2,1
4,0,1,1,0,1,0,4,3,0,1,0,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57451,4,0,1,1,0,4,1,2,3,1,1,1,1,1,0
57452,0,1,1,0,1,0,2,0,0,8,0,2,0,7,0
57453,0,1,1,0,0,0,5,5,1,1,0,2,2,2,1
57454,0,0,1,0,1,0,7,0,4,1,0,0,1,6,0


In [189]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sodapy import Socrata
# Load the data
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sodapy import Socrata
# Load the data
client = Socrata("data.sfgov.org", "7cCZMHDK3CTLNu0K31rZxP1bl", username="zzp9067788@gmail.com", password="Zzp123456.")
results = client.get("ubvf-ztfx", limit=800000)
traffic_df = pd.DataFrame.from_records(results)
# List of columns to keep
# List of columns to keep
columns_to_keep = [
    'weather_1', 'collision_severity', 'type_of_collision', 
    'road_surface', 'road_cond_1', 'lighting', 
    'control_device', 'intersection', 'number_killed', 'number_injured','time_cat','mviw','ped_action','party1_type','party2_type','party1_move_pre_acc','party2_move_pre_acc'
]

# Select only the specified columns
incident_df_subset = traffic_df[columns_to_keep]

# Now, incident_df_subset contains only the desired columns
incident_df_subset['collision_severity'].value_counts()
def remap_severity(severity):
    if severity == 'Injury (Complaint of Pain)':
        return 0  # Minor Injury
    elif severity == 'Injury (Other Visible)':
        return 1  # Moderate Injury
    else:
        return 2  # Severe Injury (including 'Injury (Severe)', 'Fatal', 'Medical')

incident_df_subset['collision_severity_class'] = incident_df_subset['collision_severity'].apply(remap_severity)
# Assuming 'incident_df_subset' is your DataFrame
# List of categorical columns to be one-hot encoded
def map_weather_condition(condition):
    if condition in ['Clear', 'Cloudy', 'Raining', 'Not Stated']:
        return condition
    elif 'Fog' in condition:
        return 'Fog'
    elif condition in ['Wind', 'Snowing', 'Other', 'Other: NOT ON SCENE', 
                        'Other: Unknown', 'Other: NOT AT SCENE', 'Other: MISTING']:
        return 'Other'
    else:
        return 'Unknown'  # Fallback category

incident_df_subset['weather_1_grouped'] = incident_df_subset['weather_1'].apply(map_weather_condition)

# Convert the grouped categories to numerical values
weather_grouped_mapping = {label: idx for idx, label in enumerate(incident_df_subset['weather_1_grouped'].unique())}
incident_df_subset['weather_1_mapped'] = incident_df_subset['weather_1_grouped'].map(weather_grouped_mapping)

intersection_mapping = {label: idx for idx, label in enumerate(incident_df_subset['intersection'].unique())}
incident_df_subset['intersection_mapped'] = incident_df_subset['intersection'].map(intersection_mapping)

road_cond_mapping = {label: idx for idx, label in enumerate(incident_df_subset['road_cond_1'].unique())}
incident_df_subset['road_cond_1_mapped'] = incident_df_subset['road_cond_1'].map(road_cond_mapping)

control_device_mapping = {label: idx for idx, label in enumerate(incident_df_subset['control_device'].unique())}
incident_df_subset['control_device_mapped'] = incident_df_subset['control_device'].map(control_device_mapping)

lighting_mapping = {label: idx for idx, label in enumerate(incident_df_subset['lighting'].unique())}
incident_df_subset['lighting_mapped'] = incident_df_subset['lighting'].map(lighting_mapping)

road_surface_mapping = {label: idx for idx, label in enumerate(incident_df_subset['road_surface'].unique())}
incident_df_subset['road_surface_mapped'] = incident_df_subset['road_surface'].map(road_surface_mapping)

type_of_collision_mapping = {label: idx for idx, label in enumerate(incident_df_subset['type_of_collision'].unique())}
incident_df_subset['type_of_collision_mapped'] = incident_df_subset['type_of_collision'].map(type_of_collision_mapping)

time_cat_mapping = {label: idx for idx, label in enumerate(incident_df_subset['time_cat'].unique())}
incident_df_subset['time_cat_mapped'] = incident_df_subset['time_cat'].map(time_cat_mapping)

party2_move_mapping = {label: idx for idx, label in enumerate(incident_df_subset['party2_move_pre_acc'].unique())}
incident_df_subset['party2_move_pre_acc_mapped'] = incident_df_subset['party2_move_pre_acc'].map(party2_move_mapping)

party1_move_mapping = {label: idx for idx, label in enumerate(incident_df_subset['party1_move_pre_acc'].unique())}
incident_df_subset['party1_move_pre_acc_mapped'] = incident_df_subset['party1_move_pre_acc'].map(party1_move_mapping)

ped_action_mapping = {label: idx for idx, label in enumerate(incident_df_subset['ped_action'].unique())}
incident_df_subset['ped_action_mapped'] = incident_df_subset['ped_action'].map(ped_action_mapping)

party1_type_mapping = {label: idx for idx, label in enumerate(incident_df_subset['party1_type'].unique())}
incident_df_subset['party1_type_mapped'] = incident_df_subset['party1_type'].map(party1_type_mapping)

party2_type_mapping = {label: idx for idx, label in enumerate(incident_df_subset['party2_type'].unique())}
incident_df_subset['party2_type_mapped'] = incident_df_subset['party2_type'].map(party2_type_mapping)

mviw_mapping = {label: idx for idx, label in enumerate(incident_df_subset['mviw'].unique())}
incident_df_subset['mviw_mapped'] = incident_df_subset['mviw'].map(mviw_mapping)

# List of mapped columns to keep
mapped_columns = [
    'weather_1_mapped', 'intersection_mapped', 'road_cond_1_mapped', 
    'control_device_mapped', 'lighting_mapped', 'road_surface_mapped', 
    'type_of_collision_mapped', 'time_cat_mapped', 'party2_move_pre_acc_mapped', 
    'party1_move_pre_acc_mapped', 'ped_action_mapped', 'party1_type_mapped', 
    'party2_type_mapped', 'mviw_mapped','collision_severity_class'
]

# Create a new DataFrame with only the mapped columns
mapped_incident_df = incident_df_subset[mapped_columns]

# Now, 'mapped_incident_df' contains only the mapped versions of your original categorical variables




A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [190]:
mapped_incident_df

,weather_1_mapped,intersection_mapped,road_cond_1_mapped,control_device_mapped,lighting_mapped,road_surface_mapped,type_of_collision_mapped,time_cat_mapped,party2_move_pre_acc_mapped,party1_move_pre_acc_mapped,ped_action_mapped,party1_type_mapped,party2_type_mapped,mviw_mapped,collision_severity_class
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,0,1,1,0,0,0,1,1,1,0,1,1,1,1,1
2,0,1,1,0,0,0,2,2,0,1,0,2,0,0,1
3,0,2,1,0,1,0,3,3,2,2,0,2,2,2,1
4,0,1,1,0,1,0,4,3,0,1,0,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57451,4,0,1,1,0,4,1,2,3,1,1,1,1,1,0
57452,0,1,1,0,1,0,2,0,0,8,0,2,0,7,0
57453,0,1,1,0,0,0,5,5,1,1,0,2,2,2,1
57454,0,0,1,0,1,0,7,0,4,1,0,0,1,6,0


In [188]:
import hopsworks
import pandas as pd
from sodapy import Socrata
from data_preprocessing import data_preprocess
api = 'HepqrjAO5PJKyUmx.qU3vOWCYlgcF71nsB5JyDXDQqZPrIJmNK43gtPZzdXM5PSwhDAuotGM26t7ujy19'
project = hopsworks.login(api_key_value = api)
fs = project.get_feature_store()
traffic_fg = fs.get_or_create_feature_group(
        name="sf_trarric_accidents",
        version=1,
        primary_key=['weather_1_mapped', 'intersection_mapped',
       'road_cond_1_mapped', 'control_device_mapped',
       'lighting_mapped', 'road_surface_mapped',
       'type_of_collision_mapped', 'time_cat_mapped',
       'party2_move_pre_acc_mapped', 'party1_move_pre_acc_mapped', 'ped_action_mapped',
       'party1_type_mapped', 'party2_type_mapped',
       'mviw_mapped','collision_severity_class'], 
        description="Incident dataset")
    
traffic_fg.insert(mapped_incident_df)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/229528
Connected. Call `.close()` to terminate connection gracefully.


Uploading Dataframe: 0.00% |          | Rows 0/57456 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: sf_trarric_accidents_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/229528/jobs/named/sf_trarric_accidents_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7efe53c62910>, None)

In [175]:
mapped_incident_df['collision_severity_class'].value_counts()

collision_severity_class
0    36555
1    16398
2     4503
Name: count, dtype: int64

In [158]:
incident_df_subset['road_cond_1_mapped'].value_counts()

road_cond_1_mapped
1    54319
4     1322
0      685
6      480
3      268
7      127
5      119
2       72
8       49
9       15
Name: count, dtype: int64

In [129]:
# Assuming 'incident_df_subset' is your DataFrame
# List of categorical columns to be one-hot encoded
categorical_columns = [
    'weather_1', 'road_cond_1', 'intersection', 
    'control_device', 'lighting', 'road_surface', 
    'type_of_collision','time_cat','party2_move_pre_acc','party1_move_pre_acc',
    'ped_action','party1_type','party2_type','mviw'
]

# Apply one-hot encoding using pandas get_dummies
incident_df_one_hot = pd.get_dummies(incident_df_subset, columns=categorical_columns)

# 'incident_df_one_hot' now contains the one-hot encoded categorical variables 
# along with the original numeric variables


In [130]:
# Dropping the 'collision_severity' column from the DataFrame
incident_df_one_hot = incident_df_one_hot.drop('collision_severity', axis=1)
incident_df_one_hot


,number_killed,number_injured,collision_severity_class,weather_1_Clear,weather_1_Cloudy,weather_1_Fog,weather_1_Fog / Visibility,weather_1_Fog / Visibility: 10 ft,weather_1_Fog / Visibility: 800 ft,weather_1_Not Stated,...,mviw_Bicycle,mviw_Fixed Object,mviw_Motor Vehicle on Other Roadway,mviw_Non-Collision,mviw_Not Stated,mviw_Other Motor Vehicle,mviw_Other Object,mviw_Parked Motor Vehicle,mviw_Pedestrian,mviw_Train
0,0,1,2,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
1,0,1,1,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,0,2,1,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
3,0,2,1,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,0,1,1,True,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57451,0,1,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
57452,0,1,0,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
57453,0,1,1,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
57454,0,1,0,True,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [131]:
incident_df_one_hot['time_cat_10:01 am to 2:00 pm'].value_counts()

time_cat_10:01 am to 2:00 pm
False    45621
True     11835
Name: count, dtype: int64

In [50]:
incident_df_train

,weather_1_mapped,collision_severity_class,type_of_collision_mapped,road_surface_mapped,road_cond_1_mapped,lighting_mapped,control_device_mapped,intersection_mapped,number_killed,number_injured
0,0,2,6,0,2,1,1,0.0,0,1
1,0,1,1,0,0,1,1,1.0,0,1
2,0,1,5,0,0,1,1,1.0,0,2
3,0,1,2,0,0,0,1,2.0,0,2
4,0,1,7,0,0,0,1,1.0,0,1
...,...,...,...,...,...,...,...,...,...,...
57451,2,0,1,3,0,1,0,0.0,0,1
57452,0,0,5,0,0,0,1,1.0,0,1
57453,0,1,3,0,0,1,1,1.0,0,1
57454,0,0,0,0,0,0,1,0.0,0,1


In [176]:

from sklearn.model_selection import train_test_split
incident_df_train = mapped_incident_df
incident_df_train = incident_df_train.dropna()
# Define features and target variable
X = incident_df_train.drop('collision_severity_class', axis=1)
y = incident_df_train['collision_severity_class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [177]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=0, n_estimators=150, criterion='entropy')
classifier.fit(X_train, y_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Evaluating the Model
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='weighted')
rec = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

model_results = pd.DataFrame([['Random Forest (n=150)', acc, prec, rec, f1]],
                             columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])
# Calculating feature importance
importance = classifier.feature_importances_

# Sort features by importance
sorted_indices = np.argsort(importance)[::-1]

# Displaying feature importance
print("Feature Importance:")
for idx in sorted_indices:
    print(f'Feature: {X.columns[idx]}, Score: {importance[idx]:.5f}')

# Display the model results
print(model_results)


Feature Importance:
Feature: party1_move_pre_acc_mapped, Score: 0.15848
Feature: time_cat_mapped, Score: 0.13741
Feature: type_of_collision_mapped, Score: 0.10934
Feature: party2_move_pre_acc_mapped, Score: 0.10445
Feature: control_device_mapped, Score: 0.06712
Feature: mviw_mapped, Score: 0.06636
Feature: lighting_mapped, Score: 0.05716
Feature: weather_1_mapped, Score: 0.05702
Feature: intersection_mapped, Score: 0.05010
Feature: ped_action_mapped, Score: 0.04724
Feature: party1_type_mapped, Score: 0.04226
Feature: road_cond_1_mapped, Score: 0.03794
Feature: party2_type_mapped, Score: 0.03653
Feature: road_surface_mapped, Score: 0.02858
                   Model  Accuracy  Precision    Recall  F1 Score
0  Random Forest (n=150)  0.617473   0.560079  0.617473   0.57519


In [178]:
print(model_results)

                   Model  Accuracy  Precision    Recall  F1 Score
0  Random Forest (n=150)  0.617473   0.560079  0.617473   0.57519


In [179]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

logreg = LogisticRegression(max_iter=2000, random_state=42)
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)
print("Logistic Regression Classification Report:\n", classification_report(y_test, logreg_pred))


Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.97      0.78      7315
           1       0.46      0.08      0.13      3270
           2       0.00      0.00      0.00       907

    accuracy                           0.64     11492
   macro avg       0.37      0.35      0.30     11492
weighted avg       0.54      0.64      0.53     11492



In [180]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
print("Random Forest Classification Report:\n", classification_report(y_test, rf_pred))


Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.85      0.75      7315
           1       0.41      0.26      0.32      3270
           2       0.18      0.05      0.08       907

    accuracy                           0.62     11492
   macro avg       0.42      0.39      0.38     11492
weighted avg       0.56      0.62      0.58     11492



In [181]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_pred = knn_model.predict(X_test)
print("KNN Classification Report:\n", classification_report(y_test, knn_pred))


KNN Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.85      0.75      7315
           1       0.40      0.26      0.31      3270
           2       0.21      0.03      0.05       907

    accuracy                           0.62     11492
   macro avg       0.43      0.38      0.37     11492
weighted avg       0.56      0.62      0.57     11492



In [182]:
from sklearn.neural_network import MLPClassifier

# Define the MLPClassifier
# Here, we are using a simple architecture with two hidden layers, each with 100 neurons
# You can experiment with different architectures and hyperparameters
mlp_model = MLPClassifier(hidden_layer_sizes=(50, 50, 50), max_iter=1000, random_state=42)

# Fit the model on the training data
mlp_model.fit(X_train, y_train)

# Predict on the test data
mlp_pred = mlp_model.predict(X_test)

# Evaluate the model
print("MLP Classification Report:\n", classification_report(y_test, mlp_pred))


MLP Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.87      0.76      7315
           1       0.43      0.26      0.32      3270
           2       0.21      0.05      0.08       907

    accuracy                           0.63     11492
   macro avg       0.44      0.39      0.39     11492
weighted avg       0.57      0.63      0.58     11492



In [4]:
api = 'UtYWT9JBE4jbsOVW.dzfTExU7QMCzzR51EADTOZCXBzl0VmgB2y012yd8nFTG6v1VHgWazdx2a2SuJAY1'
project = hopsworks.login(api_key_value = api)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/229528
Connected. Call `.close()` to terminate connection gracefully.


In [59]:
import pandas as pd
import numpy as np
import hsfs

# Assuming you have a 2D numpy array named 'my_2d_array'

# Now, connect to Hopsworks and insert the DataFrame into the feature group
project = hopsworks.login(api_key_value=api)
fs = project.get_feature_store()

# Create or get feature group
feature_group_name = "your_feature_group_name"  # Replace with your feature group name
feature_group_version = 1  # Specify the version of the feature group
feature_group = fs.get_or_create_feature_group(
    name=feature_group_name,
    version=feature_group_version,  # Replace with your primary key column name
    primary_key=['features', 'labels'], 
    description="Your feature group description",
    online_enabled=False
)

# Save the DataFrame to the feature group
feature_group.save(df)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/229528
Connected. Call `.close()` to terminate connection gracefully.


RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/229528/featurestores/229447/featuregroups). Server response: 
HTTP code: 400, HTTP reason: Bad Request, body: b'{"errorCode":270089,"usrMsg":"project: zipeizhangnew, featurestoreId: 229447","errorMsg":"The feature group you are trying to create does already exist."}', error code: 270089, error msg: The feature group you are trying to create does already exist., user msg: project: zipeizhangnew, featurestoreId: 229447